In [1]:
using LinearAlgebra, FFTW, FunctionOperators, OffsetArrays, IterativeSolvers, ToeplitzMatrices, Printf
include("helper_functions.jl")
rank_rtol = 1e-4

FFTW.set_num_threads(20)

# 1. Algorithms
## 1.1 Harmonic Mean Iterative Reweighted Least Squares (HM-IRLS)

In [2]:
function update_H!(H, σ, ϵᵏ)
    for ind in CartesianIndices(H)
        i, j = ind[1], ind[2]
        H[ind] = 1 / (max(σ[i], ϵᵏ) * max(σ[j], ϵᵏ))
    end
end

function update_dH!(dH, σ, ϵᵏ, r̃)
    for j in eachindex(dH)
        dH[j] = 1 / (max(σ[r̃+1], ϵᵏ) * max(σ[j], ϵᵏ))
    end
end

    
split(γ, r̃, d₁, d₂) = @views begin
    γ₁ = reshape(γ[1:r̃^2], r̃, r̃)
    γ₂ = reshape(γ[r̃^2+1:r̃*(r̃+d₂)], r̃, d₂)
    γ₃ = reshape(γ[r̃*(r̃+d₂)+1:r̃*(r̃+d₁+d₂)], d₁, r̃)
    γ₁, γ₂, γ₃
end

function update_𝒟⁻¹!(𝒟⁻¹, H, dH, r̃, d₁, d₂)
    𝒟⁻¹₁, 𝒟⁻¹₂, 𝒟⁻¹₃ = split(𝒟⁻¹, r̃, d₁, d₂)
    𝒟⁻¹₁ .= H
    for i in 1:d₂
        𝒟⁻¹₂[:,i] .= dH
    end
    for i in 1:d₁
        𝒟⁻¹₃[i,:] .= dH
    end
    𝒟⁻¹ .= 1 ./ 𝒟⁻¹
end

update_𝒟⁻¹! (generic function with 1 method)

In [3]:
function get_P_operator(Uᵏ, Vᵏ, Vtᵏ, tempᵈ¹ˣᵈ², r̃, d₁, d₂, dType)
    
    tempᵈ¹ˣʳ, tempʳˣᵈ² = Array{dType}(undef, d₁, r̃), Array{dType}(undef, r̃, d₂)
    
    I_VV, I_UU = Array{dType}(undef, d₂, d₂), Array{dType}(undef, d₁, d₁)
    Iᵈ¹ˣᵈ¹, Iᵈ²ˣᵈ² = Diagonal(ones(d₁)), Diagonal(ones(d₂))
    
    Pᵏ = FunctionOperator{dType}(name="Pᵏ", inDims = (r̃*(r̃+d₁+d₂),), outDims = (d₁, d₂),
        forw = (b,γ) -> begin
                γ₁, γ₂, γ₃ = split(γ, r̃, d₁, d₂)
                # According to (2.169), the equation would be:
                # Uᵏ * γ₁ * Vᵏ' + Uᵏ * γ₂' * (I - Vᵏ*Vᵏ') + (I - Uᵏ*Uᵏ') * γ₃' * Vᵏ'
                # But as the columns of γ₂ are orthogonal to the ones in Uᵏ,
                # the rows of γ₃ are orthogonal to the columns of Vᵏ,
                # the expression can be simplified:
                # (Uᵏ * γ₁ + γ₃) * Vᵏ' + Uᵏ * γ₂
                # And this is implemented avoiding array re-allocations:
                mul!(tempᵈ¹ˣʳ, Uᵏ, γ₁)
                tempᵈ¹ˣʳ .+= γ₃
                mul!(b, tempᵈ¹ˣʳ, Vtᵏ)
                mul!(tempᵈ¹ˣᵈ², Uᵏ, γ₂)
                b .+= tempᵈ¹ˣᵈ²
            end,
        backw = (γ,Φᵃy) -> begin
                γ₁, γ₂, γ₃ = split(γ, r̃, d₁, d₂)
                # Things to do:
                # γ₁ .= Uᵏ' * Φᵃy * Vᵏ
                # γ₂ .= Uᵏ' * Φᵃy * (I - Vᵏ*Vᵏ')
                # γ₃ .= (I - Uᵏ*Uᵏ') * Φᵃy * Vᵏ
                # Efficient implementation:
                I_VV .= Iᵈ²ˣᵈ² .- mul!(I_VV, Vᵏ, Vtᵏ) # same as I - Vᵏ*Vtᵏ
                I_UU .= Iᵈ¹ˣᵈ¹ .- mul!(I_UU, Uᵏ, Uᵏ') # same as I - Uᵏ*Uᵏ'
                mul!(tempᵈ¹ˣʳ, Φᵃy, Vᵏ)
                mul!(γ₁, Uᵏ', tempᵈ¹ˣʳ)
                mul!(γ₃, I_UU, tempᵈ¹ˣʳ)
                mul!(tempʳˣᵈ², Uᵏ', Φᵃy)
                mul!(γ₂, tempʳˣᵈ², I_VV)
                γ
                #vcat(vec(γ₁), vec(γ₂), vec(γ₃))
            end)
    
    Pᵏ
end

get_P_operator (generic function with 1 method)

In [4]:
function get_CG_operator(PᵃΦᵃΦP, 𝒟_weighting, tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾, r̃, d₁, d₂, dType)
    FunctionOperator{dType}(name = "CG_op", inDims = (r̃*(r̃+d₁+d₂),), outDims = (r̃*(r̃+d₁+d₂),),
        forw = (b,γ) ->  begin
            # An efficient implementation for:
            # b .= (ϵᵏ^2 * I / (𝒟⁻¹ - ϵᵏ^2 * I)) * γ + Pᵏ' * Φ' * Φ * Pᵏ * γ
            mul!(tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾, PᵃΦᵃΦP, γ)
            mul!(b, Diagonal(𝒟_weighting), γ)
            b .+= tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾
        end)
end

get_CG_operator (generic function with 1 method)

In [5]:
function HM_IRLS(
        Xᴳᵀ::AbstractArray,                     # ground truth for MSE evaluation
        y::AbstractArray,                       # under-sampled data
        Φ::FunctionOperator;                    # sampling operator
        img_size::NTuple = size(Xᴳᵀ),           # size of output matrix
        r̃::Int = 0,                             # rank estimate of solution
        maxIter::Union{Int, Nothing} = nothing, # number of CG iteration steps
        N::Int = 10,                            # number of iterations
        verbose::Bool = false)                  # print rank and loss value in each iteration

    # Initialize variables
    @assert 3 ≤ length(img_size) ≤ 4
    if length(img_size) == 3
        nx,ny,nt = img_size
        nc = size(y)[end]
    else
        nx,ny,nt,nc = img_size
    end
    dType = eltype(y)
    d₁, d₂ = nx*nx, nt
    y = reshape(y, :, nt, nc)
    Φ = reshape(Φ, inDims = (d₁, d₂), outDims = size(y))
    Xᴳᵀ = reshape(Xᴳᵀ, d₁, d₂)
    r̃ == 0 && (r̃ = rank(Xᴳᵀ))
    maxIter = maxIter isa Nothing ? r̃*(r̃+d₁+d₂) : maxIter
    ϵᵏ = Inf
    Xᵏ = Φ' * y
    
    # Preallocate arrays
    F = svd(Xᵏ)
    Uᵏ, σ, Vᵏ, Vtᵏ = F.U[:, 1:r̃], F.S, F.V[:, 1:r̃], F.Vt[1:r̃, :]
    Hᵏᵤᵥ = Array{dType}(undef, r̃, r̃)
    dHᵏ = Array{dType}(undef, r̃)
    𝒟⁻¹, 𝒟_weighting, b, γᵏ, tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾ = [Vector{dType}(undef, r̃*(r̃+d₁+d₂)) for _ in 1:5]
    tempᵈ¹ˣᵈ² = Array{dType}(undef, d₁, d₂)
    rᵏ, γᵏ_tilde = similar(y), similar(γᵏ)
    statevars = IterativeSolvers.CGStateVariables(similar(γᵏ), similar(γᵏ), similar(γᵏ))
    
    # Create operators
    Pᵏ= get_P_operator(Uᵏ, Vᵏ, Vtᵏ, tempᵈ¹ˣᵈ², r̃, d₁, d₂, dType)
    PᵃΦᵃΦP = Pᵏ' * Φ' * Φ * Pᵏ
    ΦP, PᵃΦᵃ = Φ * Pᵏ, Pᵏ' * Φ'
    CG_op = get_CG_operator(PᵃΦᵃΦP, 𝒟_weighting, tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾, r̃, d₁, d₂, dType)
    
    cost_vec = OffsetVector{real(dType)}(undef, 0:N)
    rank_vec = OffsetVector{Int}(undef, 0:N)
    time_vec = OffsetVector{Float64}(undef, 0:N)
    time_vec[0] = 0

    cost_vec[0] = norm(tempᵈ¹ˣᵈ² .= Xᴳᵀ .-  Xᵏ)
    rank_vec[0] = sum(Int, σ .> rank_rtol)
    verbose && iterationPrint("k" => 0, "rank(Xᵏ)" => rank_vec[0],
            "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[0], "σ₁" => σ[1], "ϵᵏ" => ϵᵏ)
    
    for k in 1:N
        
        time_vec[k] = time_vec[k-1] + @elapsed begin
            
            svd!(tempᵈ¹ˣᵈ² .= Xᵏ, F)
            @views begin Uᵏ .= F.U[:, 1:r̃]; Vᵏ .=  F.V[:, 1:r̃]; Vtᵏ .= F.Vt[1:r̃, :]; end

            ϵᵏ = min(ϵᵏ, σ[r̃+1])

            update_H!(Hᵏᵤᵥ, σ, ϵᵏ)
            update_dH!(dHᵏ, σ, ϵᵏ, r̃)
            update_𝒟⁻¹!(𝒟⁻¹, Hᵏᵤᵥ, dHᵏ, r̃, d₁, d₂)

            # An efficient implementation of 𝒟_weighting = ϵᵏ^2 * I / (𝒟⁻¹ - ϵᵏ^2 * I):
            𝒟_weighting .= ϵᵏ^2 ./ (𝒟⁻¹ .- ϵᵏ^2)

            mul!(b, PᵃΦᵃ, y) # right hand side for CG
            mul!(γᵏ, Pᵏ', Xᵏ) # initial value for CG

            cg!(γᵏ, CG_op, b, maxiter = maxIter, statevars = statevars) # 2.167

            # An efficient implementation of rᵏ = y - Φ * Pᵏ * γᵏ:
            rᵏ .= y .- mul!(rᵏ, ΦP,  γᵏ)

            # An efficient implementation of γᵏ_tilde = Diagonal(𝒟⁻¹ ./ (𝒟⁻¹ .- ϵᵏ^2)) * γᵏ - Pᵏ' * Φ' * rᵏ
            𝒟_weighting .= 𝒟⁻¹ ./ (𝒟⁻¹ .- ϵᵏ^2) # same as Diagonal(𝒟⁻¹ ./ (𝒟⁻¹ .- ϵᵏ^2))
            mul!(tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾, PᵃΦᵃ, rᵏ)
            mul!(γᵏ_tilde, Diagonal(𝒟_weighting), γᵏ)
            γᵏ_tilde .-= tempʳ⁽ʳ⁺ᵈ¹⁺ᵈ²⁾

            # An efficient implementation of Xᵏ = Φ' * rᵏ + Pᵏ * γᵏ_tilde
            mul!(Xᵏ, Pᵏ, γᵏ_tilde)
            Xᵏ .+= mul!(tempᵈ¹ˣᵈ², Φ', rᵏ)   # 2.168
            
        end

        cost_vec[k] = norm(tempᵈ¹ˣᵈ² .= Xᴳᵀ .-  Xᵏ)
        rank_vec[k] = sum(Int, σ .> rank_rtol)
        verbose && iterationPrint("k" => k, "rank(Xᵏ)" => rank_vec[k],
                "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[k], "σ₁" => σ[1], "ϵᵏ" => ϵᵏ)
        
    end
    
    reshape(Xᵏ, nx, ny ,nt), cost_vec, rank_vec, time_vec
end

HM_IRLS (generic function with 1 method)

The algorithms below are described in [this paper](https://web.eecs.umich.edu/~fessler/papers/files/jour/19/web/lin-19-edp.pdf) by Claire Yilin Lin, and Jeffrey A. Fessler, and follows their [original implementation in Matlab](https://github.com/JeffFessler/reproduce-l-s-dynamic-mri).

## 1.2 Augmented Lagrangian + Conjugate Gradient

**Problem formulation:** $argmin_{L,S}min_{U,W}\{\frac{1}{2}\lVert \textbf{E}(L+S) -d \lVert_2^2 + \lambda_L \lVert P \lVert_* + \lambda_S \lVert Q \lVert_1\} \textrm{ subject to } \begin{cases} P = L \\ Q = \textbf{T}S \end{cases}$

**Modified AL function:** $\frac{1}{2}\lVert \textbf{E}(L+S) - d \lVert_2^2 + \lambda_L \lVert P \lVert_* + \lambda_S \lVert Q \lVert_1\ + \frac{\delta_1}{2} \lVert L - P + V_1 \lVert_2^2 + \frac{\delta_2}{2} \lVert \textbf{T}S - Q + V_2 \lVert_2^2$

**Inputs:**

 - $d$: under-sampled multi-coil k-t data
 - $\textbf{E}$: Acquisition operator
 - $\textbf{T}$: temporal Fourier tranform
 - $\lambda_L$: singular value threshold
 - $\lambda_S$: sparsity threshold
 - $iterL$: number of conjugate gradient iteration steps for $S$
 - $iterL$: number of conjugate gradient iteration steps for $L$
 - $\delta_1$, $\delta_2$: AL penalty parameters
 - $N$: number of iterations
 
**Initialization:** $L_0 = \textbf{E}^∗ d$, $S_0 = V_{1,0} = V_{2,0} = 0$

for $k = 1, 2, ... , N$ do
 - $P_k \leftarrow \textbf{SVT}_{\lambda_L / \delta_1} (L+V_2)$
 - $Q_k \leftarrow \textbf{T}^* \Lambda_{\lambda_S / \delta_2} (\textbf{T}S) + V_2 $
 - $L_k \leftarrow CG_{solver}\{\textrm{get } x \textrm{ s.t. } (\textbf{E}^*\textbf{E} + \delta_1\textbf{I})x = x_0 - \textbf{E}^*\textbf{E}S + \delta_1(P-V_1) \textrm{ starting from }L_{k-1}\}$,<br> where $x_0 = \textbf{E}^∗ d$ and $CG_{solver}$ is an iterative solver for conjugate gradient method
 - $S_k \leftarrow CG_{solver}\{\textrm{get } x \textrm{ s.t. } (\textbf{E}^*\textbf{E} + \delta_2\textbf{I})x = x_0 - \textbf{E}^*\textbf{E}L + \delta_2(Q-V_2) \textrm{ starting from }S_{k-1}\}$,<br> where $x_0 = \textbf{E}^∗ d$ and $CG_{solver}$ is an iterative solver for conjugate gradient method
 - $V_{1,k} \leftarrow V_{1,k-1} + L_k - P$
 - $V_{2,k} \leftarrow V_{2,k-1} + \textbf{T}S_k - Q$

end for

**output:** $L_N, S_N$ 

Respective Matlab code: https://github.com/JeffFessler/reproduce-l-s-dynamic-mri/blob/master/algorithms/AL_CG.m

In [45]:
function AL_CG(
        Xᴳᵀ::AbstractArray,                   # ground truth for MSE evaluation
        d::AbstractArray,                     # under-sampled k-t data
        E::FunctionOperator,                  # acquisition operator
        T::FunctionOperator;                  # temporal Fourier tranform
        img_shape::NTuple = size(d),          # size of output image
        scale_L::Real = 1,                    # scaling factor for L
        scale_S::Real = 1,                    # scaling factor for S
        λ_L::Real = 0.01,                     # singular value threshold
        λ_S::Real = 0.05,                     # sparsity threshold
        δ₁::Real = 1//10,                     # first AL penalty parameter
        δ₂::Real = 1//100,                    # second AL penalty parameter
        iterL::Int = 3,                       # number of CG iteration steps for S
        iterS::Int = iterL,                   # number of CG iteration steps for L
        N::Int = 10,                          # number of iterations
        verbose::Bool = false)                # print rank and loss value in each iteration
    
    complexType = eltype(d)
    floatType = real(complexType)
    
    scale_L, scale_S, λ_L, λ_S, δ₁, δ₂ = convert(floatType, scale_L), convert(floatType, scale_S),
        convert(floatType, λ_L), convert(floatType, λ_S), convert(floatType, δ₁), convert(floatType, δ₂)
    
    #Initialize
    @assert 3 ≤ length(img_shape) ≤ 4
    if length(img_shape) == 3
        nx,ny,nt = img_shape
        nc = size(d)[end]
    else
        nx,ny,nt,nc = img_shape
    end
    d = reshape(d, :, nt, nc)
    E = reshape(E, inDims = (nx*nx, nt), outDims = size(d))
    Xᴳᵀ = reshape(Xᴳᵀ, nx*nx, nt)
    x₀ = E' * d # initial guess
    
    L, S = copy(x₀), zeros(complexType, size(x₀))
    V₁, V₂ = zeros(complexType, size(L)), zeros(complexType, size(L))
    
    SVT! = getSVT()
    
    cg_tol = convert(floatType, 1e-5)
    P, Q = similar(L), similar(L)
    temp₁, temp₂ = similar(L), similar(L)
    
    cost_vec = OffsetVector{floatType}(undef, 0:N)
    rank_vec = OffsetVector{Int}(undef, 0:N)
    time_vec = OffsetVector{floatType}(undef, 0:N)
    time_vec[0] = 0
    cost_vec[0] = norm(temp₁ .= Xᴳᵀ .- L .- S)
    rank_vec[0] = sum(Int, svdvals!(temp₁ .= L) .> rank_rtol)
    verbose && iterationPrint("k" => 0, "rank(Xᵏ)" => rank_vec[0],
            "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[0])
    
    EᴴE_op, Tᴴ = E'*E, T'
    cg_op₁, cg_op₂ = EᴴE_op + δ₁*I, EᴴE_op + δ₂*I
    CGstate = CGStateVariables(temp₂, similar(L), similar(L))
    
    for k in 1:N
        
        time_vec[k] = time_vec[k-1] + @elapsed begin
        
            P = SVT!(P .= L .+ V₁, scale_L * λ_L / δ₁)
            Q = Λ!(mul!(Q, T, S) .+= V₂, scale_S * λ_S / δ₂)

            temp₁ .= x₀ .- mul!(temp₁, EᴴE_op, S) .+ δ₁.*(temp₂ .= P .- V₁)
            cg!(L, cg_op₁, temp₁, tol=cg_tol, maxiter=iterL, statevars=CGstate)

            mul!(temp₂, Tᴴ, temp₁ .= Q .- V₂)
            temp₁ .= x₀ .- mul!(temp₁, EᴴE_op, L) .+ δ₂.*temp₂
            cg!(S, cg_op₂, temp₁, tol=cg_tol, maxiter=iterS, statevars=CGstate)

            V₁ .+= L .- P
            V₂ .+= mul!(temp₁, T, S) .- Q

        end
        
        cost_vec[k] = norm(temp₁ .= Xᴳᵀ .- L .- S)
        rank_vec[k] = sum(Int, svdvals!(temp₁ .= L) .> rank_rtol)
        verbose && iterationPrint("k" => k, "rank(Xᵏ)" => rank_vec[k],
                "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[k])
        
    end
    
    reshape(L, nx, ny, nt) + reshape(S, nx, ny, nt), rank_vec, cost_vec, time_vec
end

AL_CG (generic function with 1 method)

## 1.3 Improved Augmented Lagrangian

**Problem formulation:** $argmin_{L,S}min_{Z,X}\{\frac{1}{2}\lVert \boldsymbol{\Omega} Z -d \lVert_2^2 + \lambda_L \lVert L \lVert_* + \lambda_S \lVert \textbf{T}S \lVert_1\} \textrm{ subject to } \begin{cases} Z = \textbf{QC}X \\ X = L+S \end{cases}$

**Modified AL function:** $\frac{1}{2}\lVert \boldsymbol{\Omega} Z -d \lVert_2^2 + \lambda_L \lVert L \lVert_* + \lambda_S \lVert \textbf{T}S \lVert_1 + \frac{\delta_1}{2} \lVert Z - \textbf{QC} + V_1 \lVert_2^2 + \frac{\delta_2}{2} \lVert X - (L+S) + V_2 \lVert_2^2$

**Inputs:**
 - $d$: under-sampled multi-coil k-t data
 - $\boldsymbol{\Omega}$: under-sampling mask
 - $\textbf{Q}$: Fourier encoding operator
 - $\textbf{C}$: coil sensitivity maps
 - $\textbf{T}$: temporal Fourier tranform
 - $\lambda_L$: singular value threshold
 - $\lambda_S$: sparsity threshold
 - $\delta_1$, $\delta_2$: AL penalty parameters
 - $N$: number of iterations
 
**Initialization:** $X_0 = L_0 = C∗Q∗Ω∗d$, $S_0 = V_{1,0} = V_{2,0} = 0$

for $k = 1, 2, ... , N$ do
 - $Z_k \leftarrow argmin_Z \frac{1}{2} \lVert\boldsymbol{\Omega} Z-d\lVert_2^2 + \frac{\delta_1}{2}\lVert X - (L+S) + V_2\lVert_2^2 = (\boldsymbol{\Omega}^*\boldsymbol{\Omega} + \delta_1 \textbf{I})^{-1}(\boldsymbol{\Omega}^*d + \delta_1(\textbf{Q}\textbf{C}X - V_1))$
 - $X_k \leftarrow argmin_X \frac{\delta_1}{2} \lVert Z-\textbf{QC}X+V\_1 \lVert_2^2 + \frac{\delta_2}{2} \lVert X-(L+S) + V_2 \lVert_2^2$ $ = (\textbf{C}^*\textbf{C} + \frac{\delta_1}{\delta_2}\textbf{I})^{-1}(\textbf{C}^*\textbf{Q}^*(Z+V_1)+\frac{\delta_1}{\delta_2}(L+S+V_2)) = \frac{\delta_1}{\delta_1+\delta_2}(\textbf{C}^*\textbf{Q}^*(Z+V_1) + \frac{\delta_2}{\delta_1}(L+S-V_2))$
 - $L_k \leftarrow argmin_L (\lambda_L \lVert L \lVert_* + \frac{\delta_2}{2} \lVert X - (L + S) + V_2 \lVert_2^2) = \textbf{SVT}_{\lambda_L / \delta_2} (X-S+V_2)$
 - $S_k \leftarrow argmin_S (\lambda_S \lVert \textbf{T} S \lVert_1 + \frac{\delta_2}{2} \lVert X - (L + S) + V_2 \lVert_2^2)$ $ = \textbf{T}^* (argmin_S (\lambda_S \lVert \tilde{S} \lVert_1 + \frac{\delta_2}{2} \lVert \textbf{T}(X - (L + S) + V_2) - \tilde{S} \lVert_2^2)$ $ = \textbf{T}^* \Lambda_{\lambda_S / \delta_2} (\textbf{T}(X - L + V_2)) $ where $\tilde{S} = \textbf{T}S$
 - $V_{1,k} \leftarrow V_{1,k-1} + (Z_k − \textbf{QC}X_k)$
 - $V_{2,k} \leftarrow V_{2,k-1} + X_k − (L_k + S_k)$

end for

**output:** $L_N, S_N$ 

Respective Matlab code: https://github.com/JeffFessler/reproduce-l-s-dynamic-mri/blob/master/algorithms/AL_2.m

In [43]:
function AL_2(
        Xᴳᵀ::AbstractArray,                   # ground truth for MSE evaluation
        d::AbstractArray,                     # under-sampled k-t data
        Ω::FunctionOperator,                  # under-sampling mask
        Q::FunctionOperator,                  # Fourier encoding operator
        C::FunctionOperator,                  # coil sensitivity maps
        T::FunctionOperator;                  # temporal Fourier tranform
        img_shape::NTuple = size(d),          # size of output image
        scale_L::Real = 1,                    # scaling factor for L
        scale_S::Real = 1,                    # scaling factor for S
        λ_L::Real = 0.01,                     # singular value threshold
        λ_S::Real = 0.05,                     # sparsity threshold
        δ₁::Real = 1//10,                     # first AL penalty parameter
        δ₂::Real = 1//100,                    # second AL penalty parameter
        N::Int = 10,                          # number of iterations
        verbose::Bool = false)                # print rank and loss value in each iteration
    
    complexType = eltype(d)
    floatType = real(complexType)
    
    scale_L, scale_S, λ_L, λ_S, δ₁, δ₂ = convert(floatType, scale_L), convert(floatType, scale_S),
        convert(floatType, λ_L), convert(floatType, λ_S), convert(floatType, δ₁), convert(floatType, δ₂)
    
    #Initialize
    @assert 3 ≤ length(img_shape) ≤ 4
    if length(img_shape) == 3
        nx,ny,nt = img_shape
        nc = size(d)[end]
    else
        nx,ny,nt,nc = img_shape
    end
    E = reshape(Ω * Q * C, inDims = (nx*nx, nt), outDims = size(d))
    QC = reshape(Q * C, inDims = (nx*nx, nt), outDims = size(d))
    Xᴳᵀ = reshape(Xᴳᵀ, nx*nx, nt)
    Tᴴ, QCᴴ = T', QC'
    
    L = reshape(E' * d, nx*ny, nt) # initial guess
    S = zeros(complexType, size(L))
    X = L + S
    V₁ = zeros(complexType, size(d))
    V₂ = zeros(complexType, size(L))
    Z_scaler = repeat(1 ./ (samp .+ δ₁), 1, 1, 1, nc) # equivalent to (Ω'*Ω + δ₁*I)'
    temp₁, temp₂, Z, temp₃, temp₄ = similar(L), similar(L), similar(d), similar(d), similar(d)
    
    SVT! = getSVT()
    
    cost_vec = OffsetVector{floatType}(undef, 0:N)
    rank_vec = OffsetVector{Int}(undef, 0:N)
    time_vec = OffsetVector{floatType}(undef, 0:N)
    time_vec[0] = 0
    cost_vec[0] = norm(temp₁ .= Xᴳᵀ .- L .- S)
    rank_vec[0] = sum(Int, svdvals!(temp₁ .= L) .> rank_rtol)
    verbose && iterationPrint("k" => 0, "rank(Xᵏ)" => rank_vec[0],
            "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[0])
    
    # Iteration
    for k in 1:N
        
        time_vec[k] = time_vec[k-1] + @elapsed begin
        
            temp₃ .= d #mul!(temp₃, Ω', d)
            mul!(temp₄, QC, X)
            Z .= Z_scaler .* (temp₃ .+ δ₁ .* (temp₄ .- V₁))

            mul!(temp₁, QCᴴ, temp₃ .= Z .+ V₁)
            temp₂ .= L .+ S .- V₂
            X .= δ₁./(δ₁ .+ δ₂).*temp₁ .+ δ₂./(δ₁ .+ δ₂).*temp₂

            L = SVT!(L .= X .- S .+ V₂, scale_L * λ_L / δ₂)

            mul!(temp₁, T, temp₁ .= X .- L .+ V₂)
            mul!(S, Tᴴ, Λ!(temp₁, scale_S * λ_S / δ₂))

            mul!(temp₃, QC, X)
            V₁ .+= Z .- temp₃

            V₂ .+= X .- L .- S
            
        end
        
        cost_vec[k] = norm(temp₁ .= Xᴳᵀ .- L .- S)
        rank_vec[k] = sum(Int, svdvals!(temp₁ .= L) .> rank_rtol)
        verbose && iterationPrint("k" => k, "rank(Xᵏ)" => rank_vec[k],
                "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[k])
    end
    
    reshape(L, nx, ny, nt) + reshape(S, nx, ny, nt), rank_vec, cost_vec, time_vec
end

AL_2 (generic function with 1 method)

## 1.4 Proximal Gradient Methods

**Inputs:**

 - $d$: under-sampled k-t data
 - $\textbf{E}$: data acquisition operator
 - $\textbf{T}$: temporal Fourier transform
 - $\lambda_L$: singular value threshold
 - $\lambda_S$: sparsity threshold
 - $N$: number of iterations
 - $momentum$: update rule (ISTA/FISTA/POGM)
 
**Initialization:** $M_0 = L_0 = \tilde{L}_0 = \bar{L}_0 = E∗d, S_0 = 0, \theta_0 = \zeta_0 = 1, t = 0.5 \textrm{ (for FISTA and POGM) or } 0.99 \textrm{ (for ISTA)}$

for $k = 1, 2, ... , N$ do

 - $\tilde{L}_k \leftarrow M_{k-1} - S_{k-1}$
 - $\tilde{S}_k \leftarrow M_{k-1} - L_{k-1}$
 - $\theta_k \leftarrow \begin{cases} \frac{1+\sqrt{1 + 4\theta_{k-1}^2}}{2} \\ \frac{1+\sqrt{1 + 8\theta_{k-1}^2}}{2} \end{cases}$
 - $\bar{X}_k \leftarrow \begin{cases} \tilde{X}_k : momentum = ISTA \\ \tilde{X}_k + \frac{\theta_{k-1}-1}{\theta_k}(\tilde{X}_k - \tilde{X}_{k-1}) + \frac{\theta_{k-1}}{\theta_k}(\tilde{X}_k - X_{k-1}) : momentum = FISTA \\ \tilde{X}_k + \frac{\theta_{k-1}-1}{\theta_k}(\tilde{X}_k - \tilde{X}_{k-1}) + \frac{\theta_{k-1}}{\theta_k}(\tilde{X}_k - X_{k-1}) + \frac{\theta_{k-1}-1}{\zeta_{k-1} \theta_k}(\bar{X}_{k-1} - X_{k-1}) : momentum = POGM \end{cases}$
 - $\zeta_k \leftarrow t(1+\frac{\theta_{k-1}-1}{\theta_k} + \frac{\theta_{k-1}}{\theta_k})$
 - $L_k \leftarrow \textbf{SVT}_{\lambda_L}(\bar{L}_k)$
 - $S_k \leftarrow \textbf{T}^*(\Lambda_{\lambda_S}[\textbf{T}(\bar{S}_k)])$
 - $M_k \leftarrow L_k + S_k - t\textbf{E}^* (\textbf{E}(L_k +S_k)-d)$

end for

**output:** $X_N$

This funciton creates the following construct: $X = \begin{bmatrix}L \\ S\end{bmatrix}$ in such a way that modifying $L$ or $S$ also takes effect in $X$, and vice versa.

In [8]:
function createXLS(dType, row_dim, nt)
    X = zeros(dType, row_dim, 2*nt)
    L = view(X, :, 1:nt)
    S = view(X, :, nt + 1:2*nt)
    return X,L,S
end
function allocate(dType, row_dim, nt)
    createXLS(dType, row_dim, nt)..., createXLS(dType, row_dim, nt)...
end

allocate (generic function with 1 method)

Respective Matlab code: https://github.com/JeffFessler/reproduce-l-s-dynamic-mri/blob/master/algorithms/PGM.m

In [48]:
function PGM(
        Xᴳᵀ::AbstractArray,                 # ground truth for MSE evaluation
        d::AbstractArray,                   # under-sampled k-t data
        E::FunctionOperator,                # acquisition operator
        T::FunctionOperator;                # sparsifying operator
        img_shape::NTuple = size(d),        # size of output image
        scale_L::Real = 1,                  # scaling factor for L
        scale_S::Real = 1,                  # scaling factor for S
        tscale::Real = 1,                   # scaling factor for t
        λ_L::Real = 0.01,                   # singular value threshold
        λ_S::Real = 0.05,                   # sparsity threshold
        N::Int = 10,                        # number of iterations
        restart::Bool = true,               # reset θₖ if cost increased
        momentum::Symbol = :pogm,           # update rule (:pogm, :ista, :fista)
        verbose::Bool = false)              # print rank and loss value in each iteration
    
    complexType = eltype(d)
    floatType = real(complexType)
    
    scale_L, scale_S, tscale, λ_L, λ_S = convert(floatType, scale_L), convert(floatType, scale_S),
        convert(floatType, tscale), convert(floatType, λ_L), convert(floatType, λ_S)
    
    #Initialize
    @assert 3 ≤ length(img_shape) ≤ 4
    if length(img_shape) == 3
        nx,ny,nt = img_shape
        nc = size(d)[end]
    else
        nx,ny,nt,nc = img_shape
    end
    row_dim = nx*ny
    d = reshape(d, :, nt, nc)
    E = reshape(E, inDims = (row_dim, nt), outDims = size(d))
    Xᴳᵀ = reshape(Xᴳᵀ, nx*nx, nt)
    x₀ = E' * d # initial guess
    
    Xₖ₋₁,Lₖ₋₁,Sₖ₋₁,Xₖ,Lₖ,Sₖ = allocate(complexType, row_dim, nt)
    X̃ₖ₋₁,L̃ₖ₋₁,S̃ₖ₋₁,X̃ₖ,L̃ₖ,S̃ₖ = allocate(complexType, row_dim, nt)
    if momentum == :pogm
        X̄ₖ₋₁,L̄ₖ₋₁,S̄ₖ₋₁,X̄ₖ,L̄ₖ,S̄ₖ = allocate(complexType, row_dim, nt)
    else
        X̄ₖ₋₁,L̄ₖ₋₁,S̄ₖ₋₁,X̄ₖ,L̄ₖ,S̄ₖ = X̃ₖ₋₁,L̃ₖ₋₁,S̃ₖ₋₁,X̃ₖ,L̃ₖ,S̃ₖ
    end
    Lₖ₋₁ .= x₀
    X̄ₖ₋₁ .= X̃ₖ₋₁ .= Xₖ₋₁
    temp₁, temp₂ = similar(Lₖ₋₁), similar(d)
    Eᴴ, Tᴴ = E', T'
    
    t = (in(momentum, (:fista, :pogm)) ? 5//10 : 99//100) * tscale
    
    mul!(temp₂, E, x₀)
    mul!(temp₁, E', temp₂ .-= d)
    Mₖ = Mₖ₋₁ = x₀ .- t .* temp₁ # we don't need two arrays for M, but it looks better this way
    
    θₖ₋₁ = ζₖ₋₁ = 1.
    
    SVT! = getSVT()
    
    cost_vec = OffsetVector{floatType}(undef, 0:N)
    rank_vec = OffsetVector{Int}(undef, 0:N)
    time_vec = OffsetVector{floatType}(undef, 0:N)
    time_vec[0] = 0
    cost_vec[0] = norm(temp₁ .= Xᴳᵀ .- Lₖ₋₁ .- Sₖ₋₁)
    rank_vec[0] = sum(Int, svdvals!(temp₁ .= Lₖ₋₁) .> rank_rtol)
    verbose && iterationPrint("k" => 0, "rank(Xᵏ)" => rank_vec[0],
            "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[0])
    
    # Iteration
    for k in 1:N
        
        time_vec[k] = time_vec[k-1] + @elapsed begin
            
            @. L̃ₖ = Mₖ₋₁ - Sₖ₋₁
            @. S̃ₖ = Mₖ₋₁ - Lₖ₋₁

            θₖ  = (1 + √(1 + (k < N ? 4 : 8)*θₖ₋₁^2))/2

            if momentum == :pogm
                @. X̄ₖ = X̃ₖ + (θₖ₋₁-1)/θₖ*(X̃ₖ - X̃ₖ₋₁) +
                    (θₖ₋₁)/θₖ*(X̃ₖ - Xₖ₋₁) + (θₖ₋₁-1)/(ζₖ₋₁*θₖ)*t*(X̄ₖ₋₁ - Xₖ₋₁)
            elseif momentum == :fista
                @. X̄ₖ = X̃ₖ + (θₖ₋₁-1)/θₖ*(X̃ₖ - X̃ₖ₋₁)
            else
                # nothing to do as X̄ₖ == X̃ₖ
            end

            ζₖ  = t*(1 + (θₖ₋₁-1)/θₖ + (θₖ₋₁)/θₖ)

            SVT!(Lₖ .= L̄ₖ, scale_L*λ_L)

            mul!(temp₁, T, temp₁ .= S̄ₖ) # T operator in doesn't like subarrays
            mul!(temp₁, Tᴴ, Λ!(temp₁, scale_S*λ_S))
            Sₖ .= temp₁

            mul!(temp₂, E, temp₁ .= Lₖ .+ Sₖ)
            mul!(temp₁, Eᴴ, temp₂ .-= d)
            @. Mₖ = Lₖ + Sₖ - t * temp₁
        
        end
        
        cost_vec[k] = norm(temp₁ .= Xᴳᵀ .- Lₖ .- Sₖ)
        rank_vec[k] = sum(Int, svdvals!(temp₁ .= Lₖ) .> rank_rtol)
        verbose && iterationPrint("k" => k, "rank(Xᵏ)" => rank_vec[k],
                "‖Xᴳᵀ - Xᵏ‖₂" => cost_vec[k])
        
        # Move (k) -> (k-1), and avoid allocation for new (k)
        #      => switch (k) and (k-1) matrices
        @swap(Xₖ₋₁, Xₖ); @swap(X̃ₖ₋₁, X̃ₖ); @swap(X̄ₖ₋₁, X̄ₖ)
        @swap(Lₖ₋₁, Lₖ); @swap(L̃ₖ₋₁, L̃ₖ); @swap(L̄ₖ₋₁, L̄ₖ)
        @swap(Sₖ₋₁, Sₖ); @swap(S̃ₖ₋₁, S̃ₖ); @swap(S̄ₖ₋₁, S̄ₖ)
        @swap(Mₖ₋₁, Mₖ)
        θₖ₋₁, ζₖ₋₁ = θₖ, ζₖ
    end
    
    reshape(Lₖ, nx, ny, nt) + reshape(Sₖ, nx, ny, nt), rank_vec, cost_vec, time_vec
end

PGM (generic function with 1 method)

# 2. Helper functions
**Thresholding**

In [15]:
pos(x) = x < 0 ? zero(x) : x
Λ! = (v,p) -> @. v = sign(v) * pos(abs(v) - p)

function getSVT()
    F = nothing
    (A,p) -> begin
        F isa Nothing ? (F = svd!(A)) : svd!(A, F)
        mul!(A, F.U, mul!(F.Vt, Diagonal(Λ!(F.S, p)), F.Vt))
    end
end

getSVT (generic function with 1 method)

**Norms**

In [16]:
normₙ!(A) = sum(svdvals!(A))

normₙ! (generic function with 1 method)

In [17]:
import Base.size
function Base.size(FO::FunctionOperator, d::Int)
    @assert d in [1, 2]
    prod(d == 1 ? FO.outDims : FO.inDims)
end

In [18]:
format(value) = value isa Real && !(value isa Int) ? @sprintf("%.3f", value) : string(value)

iterationPrint(pairs...) =
    println(join([key*" = "*format(value) for (key,value) in pairs], ",\t"))

iterationPrint (generic function with 1 method)